In [ ]:
import utils_io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
path_train_data = "data_set/train.csv"

train_data = utils_io.load_csv_data(path_train_data)
# Debugging: Check the type of train_data

# Print the loaded data
if isinstance(train_data, pd.DataFrame):
    print(f"dimension train data: {train_data.shape}")
else:
    print("train_data is not a DataFrame")

In [ ]:
printdata = train_data.iloc[:10]
print(train_data.shape)
print(printdata)

In [ ]:
# Get target data
target_data = train_data["SalePrice"]
print(target_data.shape)


# Drop the Id and sales price column
train_data = train_data.drop(columns=["Id"])
train_data = train_data.drop(columns=["SalePrice"])
print(train_data.shape)

# Normalization


Prepare data description auto generate json file


In [ ]:
import pandas as pd
import numpy as np


def auto_clean_columns(df: pd.DataFrame) -> pd.DataFrame:
    df_cleaned = df.copy()

    for column in df_cleaned.columns:
        values = df_cleaned[column].dropna().unique()

        # Pokud vše je číslo nebo 'NA' → převedeme na float
        can_be_numeric = True
        for v in values:
            if isinstance(v, str) and v.strip().upper() == "NA":
                continue
            try:
                float(v)
            except:
                can_be_numeric = False
                break

        if can_be_numeric:
            # Převést "NA" na np.nan a sloupec na float
            df_cleaned[column] = df_cleaned[column].replace("NA", np.nan)
            df_cleaned[column] = pd.to_numeric(df_cleaned[column], errors="coerce")
        else:
            # ponecháme jako kategorii se stringy
            df_cleaned[column] = df_cleaned[column].astype(str).str.strip().str.upper()

    return df_cleaned


train_data = auto_clean_columns(train_data)
print(train_data)

In [ ]:
result = {}

# column = "MSSubClass"
column = "LotFrontage"

result[column] = {"description": "", "items": {}}

id_counter = 0
seen = {}

for value in train_data[column]:
    if isinstance(value, str):
        # Add NA for first every column
        if "NA" not in seen:
            result[column]["items"]["NA"] = {"id": id_counter}
            seen["NA"] = True
            id_counter += 1

        key = value.strip().upper()
        if key not in seen:
            result[column]["items"][key] = {"id": id_counter}
            seen[key] = True
            id_counter += 1
    # else:
    # print(column)
print(result)

In [ ]:
import pandas as pd
import json


def auto_generate_description_dict(train_data: pd.DataFrame) -> dict:
    result = {}

    for column in train_data.columns:
        result[column] = {"description": "", "items": {}}

        id_counter = 0
        seen = {}

        for value in train_data[column]:
            if isinstance(value, str):
                # Add NA for first every column
                if "NA" not in seen:
                    result[column]["items"]["NA"] = {"id": id_counter}
                    seen["NA"] = True
                    id_counter += 1

                key = value.strip().upper()
                if key not in seen:
                    result[column]["items"][key] = {"id": id_counter}
                    seen[key] = True
                    id_counter += 1
            # else:
            # print(column)

    return result


#  Create descriptions from train data (DO NOT DO IT FROM TEST DATA)
description_dictionary = auto_generate_description_dict(train_data)
print(description_dictionary)

with open("data_set/generated_description.json", "w", encoding="utf-8") as f:
    json.dump(description_dictionary, f, indent=2, ensure_ascii=False)

print("✅ JSON dictionary generated.")

In [ ]:
print(train_data.iloc[:10])

In [ ]:
import json
import re

txt_path = "data_set/data_description.txt"
json_path = "data_set/generated_description.json"


# Použití
# utils_io.parse_description_txt_to_json(txt_path, json_path)

generated_description = json.load(open(json_path))


for column in train_data.columns:
    for index, value in train_data[column].items():
        # if value == "C (all)":
        #     print(column, index, value)
        if isinstance(value, str):
            try:
                train_data.at[index, column] = generated_description[column]["items"][
                    value.strip().upper()
                ]["id"]
            except:
                print(column, index, value, isinstance(value, str))

In [ ]:
print(train_data.iloc[:10])

calc mean and std


In [ ]:
mean = train_data.mean(axis=0)
# print(mean)
std = train_data.std(axis=0)
# print(std)

train_data = train_data - mean
train_data = train_data / std

In [ ]:
print(train_data.iloc[:10])

# Create NN neural network


In [ ]:
print(max(target_data))
print(min(target_data))
print(np.mean(target_data))
print(max(target_data) - min(target_data))

In [ ]:
print(train_data.shape)

In [ ]:
train_data = train_data.astype("float32")
target_data = target_data.astype("float32")

In [ ]:
i = 0

k = 4
num_val_samples = len(train_data) // k
print(num_val_samples)
print(f"processing fold # {i}")
print(target_data.shape)
val_data = train_data[i * num_val_samples : (i + 1) * num_val_samples]
val_targets = target_data[i * num_val_samples : (i + 1) * num_val_samples]
print(val_data.shape)
print(val_targets.shape)

partial_train_data = pd.concat(
    [train_data[: i * num_val_samples], train_data[(i + 1) * num_val_samples :]]
)
partial_train_targets = pd.concat(
    [target_data[: i * num_val_samples], target_data[(i + 1) * num_val_samples :]]
)
print(partial_train_data.shape)
print(partial_train_targets.shape)
# print(partial_train_data.shape)
print(train_data.shape[1])

In [ ]:
baseline_prediction = np.full_like(target_data, np.mean(target_data))
baseline_mae = np.mean(np.abs(target_data - baseline_prediction))
print(baseline_mae)

In [ ]:
import keras
import time

start = time.time()


def build_model():
    input_shape = (79,)  # 79 parameters of house
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(128, activation="relu", input_shape=input_shape))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Dense(64, activation="relu"))
    model.add(keras.layers.Dense(1))  # regresní výstup
    model.compile(optimizer="adam", loss="mse", metrics=["mae"])
    return model


# def build_model():
#     model = keras.models.Sequential()
#     input_shape = (79,)  # 79 parameters of house
#     model.add(
#         keras.layers.Dense(units=256, activation="relu"),
#     )
#     model.add(keras.layers.Dense(units=256, activation="relu"))
#     model.add(keras.layers.Dense(units=256, activation="relu"))
#     model.add(keras.layers.Dense(units=1))  # output layer price of house
#     model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
#     return model


k = 5
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []
all_histories = []

for i in range(k):
    print(f"processing fold # {i}")
    val_data = train_data[i * num_val_samples : (i + 1) * num_val_samples]
    val_targets = target_data[i * num_val_samples : (i + 1) * num_val_samples]
    partial_train_data = pd.concat(
        [train_data[: i * num_val_samples], train_data[(i + 1) * num_val_samples :]]
    )
    partial_train_targets = pd.concat(
        [target_data[: i * num_val_samples], target_data[(i + 1) * num_val_samples :]]
    )

    model = build_model()
    print("- 🧠 Spouštím trénování...")
    history = model.fit(
        partial_train_data,
        partial_train_targets,
        epochs=num_epochs,
        batch_size=64,
        verbose=0,
    )

    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    print(f"- ✅ Fold {i + 1} finished: val_mae = {val_mae:.2f}")
    all_scores.append(val_mae)
    all_histories.append(history)

print(all_scores)
print(f"🕒 Celkový čas trénování: {time.time() - start:.2f} s")

In [ ]:
model.save("house_prices_0.keras")

In [ ]:
print("\n" + "=" * 50)
print("📊 Výsledky cross-validace:")
for i, score in enumerate(all_scores):
    print(f"Fold {i + 1}: MAE = {score:.2f}")

average = np.mean(all_scores)
print("-" * 50)
print(f"📈 Průměrná MAE přes {k} foldů: {average:.2f}")

In [ ]:
epoch_maes = [
    [history.history["mae"][epoch] for history in all_histories]
    for epoch in range(num_epochs)
]
average_mae_history = [np.mean(epoch) for epoch in epoch_maes]
std_mae_history = [np.std(epoch) for epoch in epoch_maes]

plt.figure(figsize=(10, 6))
plt.plot(
    range(1, num_epochs + 1), average_mae_history, marker="o", label="Průměrná MAE"
)
plt.fill_between(
    range(1, num_epochs + 1),
    np.array(average_mae_history) - np.array(std_mae_history),
    np.array(average_mae_history) + np.array(std_mae_history),
    alpha=0.2,
    label="± 1 std",
)
plt.xlabel("Epoch")
plt.ylabel("Mean Absolute Error (MAE)")
plt.title("📈 Průměrná MAE s rozptylem (všechny foldy)")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

# Test model on test data
